In [1]:
%load_ext autoreload
%autoreload 2

%cd ../../
!hostname

/p/fastdata/pli/Private/oberstrass1/datasets/vervet1818-3d
jrlogin04.jureca


In [50]:
import os
import re

import numpy as np
import pandas as pd
import SimpleITK as sitk

import pli

from vervet1818_3d.utils.transforms import load_disp_field

In [39]:
image_path = "datasets/vervet1818-stitched/data/aa/ntransmittance/"
trans_path = "data/aa/transformation/"

###

feature_path = "data/aa/features/resnet50_planes8_overlap/"
mask_path = "data/aa/masks/cortex/"

###



p = re.compile('Vervet1818([a-z]*)\_.*s([0-9]{4})\_.*')

image_list = []
for f in sorted(os.listdir(image_path)):
    id = int(p.match(f)[2])
    roi = p.match(f)[1]
    image_list.append({'id': id, 'roi': roi, 'file': os.path.join(image_path, f)})
image_df = pd.DataFrame(image_list).sort_values('id').reset_index(drop=True)

trans_list = []
for f in sorted(os.listdir(trans_path)):
    id = int(p.match(f)[2])
    roi = p.match(f)[1]
    trans_list.append({'id': id, 'roi': roi, 'file': os.path.join(trans_path, f)})
trans_df = pd.DataFrame(trans_list).sort_values('id').reset_index(drop=True)

files_df = trans_df.merge(image_df, on=['id', 'roi'], how='inner', suffixes=('_trans', '_image'))

print(f"Found {len(files_df)} pairs")
files_df.head()

Found 234 pairs


,id,roi,file_trans,file_image
0,841,aa,data/aa/transformation/Vervet1818aa_60mu_70ms_...,datasets/vervet1818-stitched/data/aa/ntransmit...
1,842,aa,data/aa/transformation/Vervet1818aa_60mu_70ms_...,datasets/vervet1818-stitched/data/aa/ntransmit...
2,843,aa,data/aa/transformation/Vervet1818aa_60mu_70ms_...,datasets/vervet1818-stitched/data/aa/ntransmit...
3,844,aa,data/aa/transformation/Vervet1818aa_60mu_70ms_...,datasets/vervet1818-stitched/data/aa/ntransmit...
4,845,aa,data/aa/transformation/Vervet1818aa_60mu_70ms_...,datasets/vervet1818-stitched/data/aa/ntransmit...


In [77]:
ix = 0

###

test_row = files_df.loc[ix]

# Load image
test_section = pli.data.Section(test_row.file_image)
test_image = sitk.GetImageFromArray(test_section.image[:])
test_image.SetSpacing(tuple(s / 1000 for s in test_section.spacing[::-1]))
print("Load image with size", test_image.GetSize(), "and spacing", test_section.spacing)

# Load transformation
test_trans, test_spacing = load_disp_field(test_row.file_trans)
test_spacing = tuple(t * 1000 for t in test_spacing)
print("Load transformation with spacing", test_spacing)

Load image with size (20079, 28852) and spacing (1.3184, 1.3184)
Load transformation with spacing (42.18880087137222, 42.18880087137222)


In [78]:
# Warp image

out_size = (31076, 28721)  # width, height
out_spacing = tuple(s / 1000 for s in test_section.spacing[::-1])
out_origin = (0, 0)

###

# Resample image
resampler = sitk.ResampleImageFilter()
resampler.SetSize(out_size)
resampler.SetOutputSpacing(out_spacing)
resampler.SetTransform(test_trans)

out_image = resampler.Execute(test_image)

In [79]:
# Save image

out_file = "data/test.h5"
chunk_size = 256
num_levels = 11


###

test_section.image = sitk.GetArrayFromImage(out_image)
test_section.to_hdf5(
    out_file,
    chunk_size=chunk_size,
    num_levels=num_levels
)

/p/project/cjinm11/Private/oberstrass1/git/envs/pli-env/venv/lib/python3.9/site-packages/atlaslib/image.py:497: UserWarning: level 0 is already located in pyramid path. Not linking again.
  warnings.warn("level 0 is already located in pyramid path. Not linking again.")
